# Ejemplo Web scrapping 2: Noticias deportivas

En este notebook vamos a ver otro ejemplo de web scrapping, este con noticias deportivas:

## Importando librerías

Al igual que en el notebook anterior, importamos lo necesario:

In [101]:
from bs4 import BeautifulSoup
from requests import get
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()


from urllib.request import urlopen

Leemos la web:

In [102]:
# Cabeceras:
headers = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})
# url web a scrapear:
marca = "https://www.marca.com/"

response = get(marca, headers = headers)

Hacemos el get:

In [104]:
print(response) # un 200 es una buena señal :D

<Response [200]>


Todo bien, pasamos a sacar cosas:

In [108]:
html_soup = BeautifulSoup(response.text, 'html.parser')

Tras enalizar la web, nos hemos fijado que todas las noticias tienen etiquetas ``h2`` y clase ``mod-title``:

In [118]:
noticias = html_soup.find_all('h2', class_='mod-title')
noticias[0]

<h2 class="mod-title" itemprop="headline"><a href="https://www.marca.com/otros-deportes/2021/01/16/6001728222601d96068b45b2.html" itemprop="url" title="El 'club del caníbal' espera a Cristiano Ronaldo"> El 'club del caníbal' espera a Cristiano Ronaldo
</a></h2>

Dentro de cada una de ellas, tenemos que extraer, a su vez, el ``title`` de las ``a``:

In [148]:
noticia_ejemplo = noticias[0].find_all("a")[0]
# Con el strip eliminamos todas aquellas cosas raras al comienzo y final del string:
noticia_ejemplo.text.strip()

"El 'club del caníbal' espera a Cristiano Ronaldo"

También podemos sacar el enlace como:

In [151]:
noticia_ejemplo.get_attribute_list('href')[0]

'https://www.marca.com/otros-deportes/2021/01/16/6001728222601d96068b45b2.html'

Ahora podemos acceder a ese link y extraer la información de la noticia en sí:

In [219]:
url_2 = noticia_ejemplo.get_attribute_list('href')[0]

response_2 = get(url_2, headers = headers)
html_soup_2 = BeautifulSoup(response_2.text, 'html.parser')


# for texto in html_soup_2.find_all("p"):
#     print(texto.text)

# El texto que queremos está en los p y en los h3, pero también se nos está colando cosas que no nos resultan neceasrias, ¿cómo podemos proceder?
# Pues leyendo todo junto (que estará en orden según la página) y quedándonos con el texto hasta que detectemos algo que nos haga parar.
# En este caso, ese algo será cuando lleguemos a un texto 'h3' que diga "Temas relaciondos". A partir de ese momento, ya no nos interesan 
#ni los 'p' ni los 'h3'
texto_noticia = html_soup_2.find_all(["p", "h3"])
texto_noticia

noticia_completa = []
cont = 0
for line in texto_noticia:
    cont += 1
    if line.text == 'Temas relacionados':
        break
    # Nos saltamos siempre las 2 primeras líneas:
    if cont > 2:
        noticia_completa.append(line.text.strip())

# noticia_completa
# Y lo juntamos todo en un solo texto:
''.join(noticia_completa)

'La desgracia se ceba con Karl-Anthony TownsRicky Rubio y Juancho Hernangómez, en cuarentenaLos contagios de la covid-19 sigue haciendo estragos dentro de la NBA. Pero especialmente con jugador que se ha visto afectado: el pívot estadounidense-dominicano Karl-Anthony Towns, de los Minnesota Timberwolves, quien confirmó a través de la redes sociales que había dado positivo."Antes del partido de esta noche, recibí otra llamada terrible y di positivo por COVID. Me aislaré inmediatamente y seguiré todos los protocolos", afirmaba el jugador de origen dominicano que ha perdido siete familiares por culpa de la pandemia que ha costado la vida a más de dos millones de personas en todo el mundo, incluida su madre.pic.twitter.com/bk0tzC6jiV"Rezo todos los días para que esta pesadilla del virus desaparezca y les ruego a todos que continúen tomándoselo en serio y tomando todas las precauciones necesarias. No podemos parar la expansión del virus solos. Tiene que ser un esfuerzo de grupo de todos nos

In [218]:
texto_noticia

[<p class="edition-selector__menu-title">Ediciones:</p>,
 <p class="tab-news-title js-newsTitle">Baloncesto/nba</p>,
 <p class="summary-lead second izquierda"><i class="item-sumary-lead"></i><a href="https://www.marca.com/baloncesto/nba/2020/12/05/5fcb4346ca4741705e8b45b0.html">La desgracia se ceba con Karl-Anthony Towns</a></p>,
 <p class="summary-lead second izquierda"><i class="item-sumary-lead"></i><a href="https://www.marca.com/baloncesto/nba/2021/01/15/6000e9fb46163fc0bf8b4571.html">Ricky Rubio y Juancho Hernangómez, en cuarentena</a></p>,
 <p><span class="capital-letter">L</span>os contagios de la <strong><a href="https://www.marca.com/baloncesto/nba/2021/01/12/5ffe0188e2704e0d298b4584.html">covid-19</a></strong> sigue haciendo estragos dentro de la NBA. Pero especialmente con jugador que se ha visto afectado: el pívot estadounidense-dominicano <strong><a href="https://www.marca.com/baloncesto/nba/2020/12/05/5fcb4346ca4741705e8b45b0.html">Karl-Anthony Towns</a></strong>, de los 

Iteramos con esta condición. Y para combinarlo con lo otro, podríamos extraer todos los links a noticias y, posteriormente, extraer el texto de cada una de ellas:

In [178]:
noticias[0].find_all("a")[0].text.strip()

"El 'club del caníbal' espera a Cristiano Ronaldo"

In [220]:
# Leemos títulos de artículos y sus links y los guardamos:
marca = "https://www.marca.com/"

response = get(marca, headers = headers)

html_soup = BeautifulSoup(response.text, 'html.parser')

noticias = html_soup.find_all('h2', class_='mod-title')

# Noticia a noticia para sacar links:
textos = []
links = []
for noticia in noticias:
    try:
        text_noticia = noticia.find_all("a")[0].text.strip()
        link = noticia.find_all("a")[0].get_attribute_list('href')[0]
        
        textos.append(text_noticia)
        links.append(link)
    except:
        pass
    
# Una vez tenemos las noticias, iteramos por cada uno de sus enlaes:
textos_noticias_completos = []
for link in links:
    response_2 = get(link, headers = headers)
    html_soup_2 = BeautifulSoup(response_2.text, 'html.parser')
    
    texto_noticia = html_soup_2.find_all(["p", "h3"])
    texto_noticia
    noticia_completa = []
    cont = 0
    for line in texto_noticia:
        cont += 1
        if line.text == 'Temas relacionados':
            break
        # Nos saltamos siempre las 2 primeras líneas:
        if cont > 2:
            noticia_completa.append(line.text.strip())
    
    textos_noticias_completos.append(''.join(noticia_completa))

Finalmente, nos creamos el dataframe con todo lo que tenemos.

Importante, si vamos a manejar muchos datos (pero muchos muchos), será mejor ir creando el dataframe poco a poco (cada x llamadas) para no almacenar demasiada información duplicada al mismo tiempo.

In [222]:
cols = ['Título', 'Descripción', 'Enlace']

noticias_deportivas = pd.DataFrame({'Título': textos,
                                    'Descripción': textos_noticias_completos,
                                    'Enlace': links})[cols]

noticias_deportivas.to_excel('marca_noticias.xls', index=False)
noticias_deportivas_df = pd.read_excel('marca_noticias.xls')

In [223]:
noticias_deportivas_df

,Título,Descripción,Enlace
0,El 'club del caníbal' espera a Cristiano Ronaldo,Cristiano Ronaldo superó a Pelé como segundo m...,https://www.marca.com/otros-deportes/2021/01/1...
1,Desgarrador Karl-Anthony Towns tras dar positi...,La desgracia se ceba con Karl-Anthony TownsRic...,https://www.marca.com/baloncesto/nba/2021/01/1...
2,"""La barriga de Harden ridiculiza a la NBA""",James Harden lo consiguió. No quería jugar en ...,https://www.marca.com/baloncesto/nba/opinion/2...
3,El hombre de las finales que puede perder su r...,Messi no se entrena y su participación en la f...,https://www.marca.com/futbol/barcelona/2021/01...
4,"""¿Rooney, el mejor jugador inglés? No metan a ...","Wayne Rooney es, en mi opinión y en la de much...",https://www.marca.com/futbol/premier-league/op...
5,El Mercedes Clase G encara la parte final del ...,El Dakar Classic 2021 sigue avanzando y se enc...,https://www.marca.com/uestudio/2021/01/14/6000...
6,"""En mi debut grité a Cannavaro: ¡Hazme la cobe...","Pepe, el mítico central del Real Madrid, ha he...",https://www.marca.com/futbol/real-madrid/2021/...
7,La afición vaticina el fin del ciclo de Zidane,Zidane agota su once y lo paga en la Supercopa...,https://www.marca.com/futbol/real-madrid/2021/...
8,"Las portadas: el tren de Hazard, la intención ...",NaN,https://www.marca.com/mundo-marca/album/2021/0...
9,Pedrerol analiza las retransmisiones del Madri...,"Klopp: ""Bruno Fernandes ha sido un gran fichaj...",https://videos.marca.com/v/0_1nz90mgg-pedrerol...
